In [1]:
import project_diff as project_diff


In [2]:
oldprojectfile = 'sample_files/ISL69269-0 0x60e.txt'
newprojectfile = 'sample_files/ISL69269-0 0x60f.txt'
outputfilename = 'diff_file_ef.txt'

In [3]:

diff = project_diff.file_delta(oldprojectfile,newprojectfile,outputfilename)

In [4]:
diff.delta

,line,name,value,address
1008,136,rGUI5,A1A08E17,003F
1009,559,svidIccMax0,00000096,E421


In [5]:
diff.write_outputfile()